In [408]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Importing data and searching data for insights

In [3]:
faults = pd.read_csv("../data/J1939Faults.csv", low_memory = False)
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")
sfc = pd.read_excel("../data/Service Fault Codes_1_0_0_167.xlsx")

C:\ProgramData\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [4]:
faults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187335 entries, 0 to 1187334
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   RecordID               1187335 non-null  int64  
 1   ESS_Id                 1187335 non-null  int64  
 2   EventTimeStamp         1187335 non-null  object 
 3   eventDescription       1126490 non-null  object 
 4   actionDescription      0 non-null        float64
 5   ecuSoftwareVersion     891285 non-null   object 
 6   ecuSerialNumber        844318 non-null   object 
 7   ecuModel               1122577 non-null  object 
 8   ecuMake                1122577 non-null  object 
 9   ecuSource              1187335 non-null  int64  
 10  spn                    1187335 non-null  int64  
 11  fmi                    1187335 non-null  int64  
 12  active                 1187335 non-null  bool   
 13  activeTransitionCount  1187335 non-null  int64  
 14  faultValue        

Found that 'Condition Exists Engine Protection Torque Derate' is the event that occurs with spn 1569

In [6]:
faults.loc[faults['spn'] == 1569]['eventDescription'].unique()

array(['Condition Exists Engine Protection Torque Derate'], dtype=object)

In [7]:
faults.loc[faults['spn'] == 5246]['eventDescription'].unique()

array([nan], dtype=object)

No event is associated with spn 5246, full derate

In [9]:
fmi = faults.loc[faults['eventDescription'] == 'Condition Exists Engine Protection Torque Derate']['fmi'].unique()
spn = faults.loc[faults['eventDescription'] == 'Condition Exists Engine Protection Torque Derate']['spn'].unique()

print(fmi, spn)

[31] [1569]


In [10]:
faults.loc[faults['eventDescription'] == 'Condition Exists Engine Protection Torque Derate'].count()

RecordID                 10927
ESS_Id                   10927
EventTimeStamp           10927
eventDescription         10927
actionDescription            0
ecuSoftwareVersion       10908
ecuSerialNumber          10665
ecuModel                 10665
ecuMake                  10665
ecuSource                10927
spn                      10927
fmi                      10927
active                   10927
activeTransitionCount    10927
faultValue                   0
EquipmentID              10927
MCTNumber                10927
Latitude                 10927
Longitude                10927
LocationTimeStamp        10927
dtype: int64

10,927 partial derates and 1,195 full derates

In [12]:
(faults['spn'] == 5246).value_counts()

spn
False    1186140
True        1195
Name: count, dtype: int64

In [13]:
faults[faults['faultValue'].isna() == False]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp


In [14]:
pd.DataFrame(faults.loc[faults['spn'] == 5246]['EquipmentID'].value_counts()).reset_index()

,EquipmentID,count
0,1524,72
1,1535,54
2,1525,43
3,1539,32
4,1749,30
...,...,...
210,105360462,1
211,1696,1
212,105411041,1
213,1487,1


In [22]:
faulty_equipment = pd.DataFrame(faults.loc[faults['spn'] == 5246]['EquipmentID']).reset_index()
faulty_array = faulty_equipment['EquipmentID'].unique()

In [52]:
faulty_equipment_df = pd.DataFrame(faulty_array).reset_index()

In [26]:
fault_events = faults.loc[faults['EquipmentID'].isin(faulty_array)]['eventDescription'].unique()

In [28]:
non_fault_events = faults.loc[~faults['eventDescription'].isin(fault_events)]['eventDescription'].unique()

For each equipment that does have a full derate eventually, these are the list of error codes that occur. The list of error codes not associated with any full derate vehicle are also listed.

In [81]:
fault_events

array(['Low (Severity Low) Engine Coolant Level', nan,
       'Abnormal Update Rate Tire Location',
       'Incorrect Data Brake Signal Sensor 2',
       'Special Instructions System Diagnostic Code #1',
       'Special Instructions Wheel Sensor ABS Axle 2 Left',
       'Data May Be Invalid Relative Speed; Rear Axle #1 Left Wheel',
       'Low (Severity Medium) Engine Coolant Level',
       'Low (Severity High) Engine Coolant Level',
       'Condition Exists Cruise Control Enable Switch',
       'Low Voltage (Power Supply (obsolete))',
       'Special Instructions Wheel Sensor ABS Axle 1 Right',
       'Special Instructions Wheel Sensor ABS Axle 1 Left',
       'Not Reporting Data Front Operator Wiper Switch',
       'Incorrect Data Wheel Sensor ABS Axle 1 Left',
       'Data May Be Invalid Relative Speed; Front Axle Left Wheel',
       'Error in System Engine Exhaust Gas Recirculation (EGR) Valve 2 Control',
       'High (Severity Low) Water In Fuel Indicator',
       'Incorrect Data 

In [85]:
non_fault_events

array(['High Current Pressure Modulation Valve ABS Axle 2 Left',
       'High Current Retarder Control Relay',
       'Incorrect Data Warning Light Bulb',
       'High (Severity High) Particulate Matter Trap Monitor',
       'High (Severity High) Transmission Range High Actuator',
       'Abnormal Frequency Wheel Sensor ABS Axle 2 Left',
       'Incorrect Data Engine Ignition Coil #12',
       'High (Severity High) J1939 Network #2',
       'High (Severity High) Cruise Control Enable Switch',
       'Low Voltage (Wheel Sensor ABS Axle 1 Right)',
       'Low (Severity High) Particulate Matter Trap Monitor',
       'High (Severity High) Engine Starter Mode',
       'Data Drifted Low Parking Brake Switch',
       'Low Voltage (Sensor supply voltage 6)',
       'Low Current Engine Turbocharger 1 Compressor Outlet Temperature',
       'Data May Be Invalid Aftertreatment 1 Exhaust Gas Temperature 3',
       'Data May Be Invalid Aftertreatment 1 Exhaust Gas Temperature 1',
       'Data May Be

In [181]:
faults.loc[faults['eventDescription'].isin(fault_events)][['eventDescription', 'spn', 'fmi']].drop_duplicates().dropna()

,eventDescription,spn,fmi
0,Low (Severity Low) Engine Coolant Level,111,17
2,Incorrect Data Steering Wheel Angle,1807,2
8,High (Severity Low) Water In Fuel Indicator,97,15
14,Incorrect Data Brake Signal Sensor 1,1067,2
19,High Voltage (Fuel Level),96,3
...,...,...,...
1125066,High (Severity Medium) Engine Injector Cylinde...,653,16
1144666,Data Drifted Low Commanded Catalyst Reagent Co...,3828,21
1153239,High (Severity Medium) Aftertreatment 1 Intake...,3217,16
1170052,Not Reporting Data Engine Exhaust Gas Recircul...,2791,7


In [34]:
possible_faulty_equipment = faults.loc[faults['eventDescription'].isin(fault_events)]['EquipmentID'].unique()

In [36]:
possible_faulty_df = pd.DataFrame(possible_faulty_equipment).reset_index()

In [191]:
partial_derate_df = faults.loc[faults['EquipmentID'].isin(possible_faulty_equipment)].loc[faults['spn'] == 1569]

In [237]:
faults_slim = faults.loc[faults['EquipmentID'].isin(possible_faulty_equipment)]

Time series data where the partial and full derates of equipment are evaluated

In [498]:
partial_event = faults.loc[faults['spn'] == 1569][['EquipmentID', 'EventTimeStamp']].rename(columns = {'EventTimeStamp': 'partial_event'})

In [500]:
full_event = faults.loc[faults['spn'] == 5246][['EquipmentID', 'EventTimeStamp']].rename(columns = {'EventTimeStamp': 'full_event'})

In [502]:
events = partial_event.merge(full_event, on = 'EquipmentID', how = 'inner')

In [504]:
events['partial_event'] = pd.to_datetime(events['partial_event'])
events['full_event'] = pd.to_datetime(events['full_event'])

In [566]:
events_melted = events.melt(id_vars=['EquipmentID'], 
                    value_vars=['partial_event',	'full_event'],
                    var_name='events', 
                    value_name='time').reset_index()


In [550]:
events_sorted = events_melted.drop_duplicates().sort_values('EquipmentID').reset_index()

In [552]:
events_sorted

,index,EquipmentID,events,time
0,30745,105349576,partial_event,2019-03-18 20:40:38
1,30763,105349576,partial_event,2019-03-20 09:57:04
2,30749,105349576,partial_event,2019-03-19 05:00:55
3,58093,105349576,full_event,2018-07-10 15:43:58
4,58092,105349576,full_event,2018-07-06 09:42:48
...,...,...,...,...
5309,26287,306,partial_event,2018-09-07 06:47:05
5310,58297,306,full_event,2018-09-07 11:22:40
5311,58298,306,full_event,2018-09-09 15:03:53
5312,58300,306,full_event,2018-09-10 05:08:58


In [574]:
time_diff_df = (
    pd.DataFrame(events_sorted
                 .groupby('EquipmentID')['time']
                 .diff())
    .rename(columns = {'time': 'time_diff'})
    .reset_index()
)

In [576]:
time_diff_df

,index,time_diff
0,0,NaT
1,1,1 days 13:16:26
2,2,-2 days +19:03:51
3,3,-252 days +10:43:03
4,4,-5 days +17:58:50
...,...,...
5309,5309,-3 days +15:43:12
5310,5310,0 days 04:35:35
5311,5311,2 days 03:41:13
5312,5312,0 days 14:05:05


Here is a dataframe containing all vehicles associated with an event that is associated with a vehicle that has spn 5246

In [235]:
possible_faulty_df

,index,0
0,0,1439
1,1,1369
2,2,1674
3,3,1417
4,4,1597
...,...,...
1116,1116,2356
1117,1117,105393087
1118,1118,2380
1119,1119,2378


And here are all vehicles that are associated with spn 5246

In [54]:
faulty_equipment_df

,index,0
0,0,1395
1,1,1630
2,2,1487
3,3,1329
4,4,1419
...,...,...
210,210,2211
211,211,302
212,212,2175
213,213,1854


In [600]:
pd.set_option('display.max_columns', None)

In [606]:
faults_diagnostics = diagnostics.merge(faults_slim, left_on = 'FaultId', right_on = 'RecordID')

In [614]:
(
    faults_diagnostics
    .loc[faults_diagnostics['EquipmentID']
    .isin(faulty_array)]
    .loc[faults_diagnostics['spn'] == 5246]
    .head(50)
)

,Id,Name,Value,FaultId,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
418,419,IgnStatus,True,46,46,990931,2015-02-21 12:10:51.000,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,NaN,1395,105349612,36.065972,-86.433425,2015-02-21 12:11:27.000
419,420,LampStatus,22527,46,46,990931,2015-02-21 12:10:51.000,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,NaN,1395,105349612,36.065972,-86.433425,2015-02-21 12:11:27.000
16232,16233,IgnStatus,True,1919,1919,1007751,2015-02-22 19:44:55.000,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,NaN,1395,105349612,36.066203,-86.434814,2015-02-22 19:46:27.000
16233,16234,LampStatus,22527,1919,1919,1007751,2015-02-22 19:44:55.000,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,NaN,1395,105349612,36.066203,-86.434814,2015-02-22 19:46:27.000
17486,17487,LampStatus,1023,2059,2059,1010486,2015-02-23 04:00:21.000,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,False,1,NaN,1395,105349612,36.066666,-86.434537,2015-02-23 01:06:06.000
17735,17736,IgnStatus,False,2090,2090,1011009,2015-02-23 05:05:44.000,NaN,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,6X1u13D1500000000,CMMNS,0,5246,0,True,1,NaN,1630,105329900,40.733009,-74.087777,2015-02-23 05:08:23.000
17736,17737,FuelLtd,33470.466902374,2090,2090,1011009,2015-02-23 05:05:44.000,NaN,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,6X1u13D1500000000,CMMNS,0,5246,0,True,1,NaN,1630,105329900,40.733009,-74.087777,2015-02-23 05:08:23.000
17737,17738,EngineTimeLtd,4645.45,2090,2090,1011009,2015-02-23 05:05:44.000,NaN,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,6X1u13D1500000000,CMMNS,0,5246,0,True,1,NaN,1630,105329900,40.733009,-74.087777,2015-02-23 05:08:23.000
17738,17739,LampStatus,22527,2090,2090,1011009,2015-02-23 05:05:44.000,NaN,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,6X1u13D1500000000,CMMNS,0,5246,0,True,1,NaN,1630,105329900,40.733009,-74.087777,2015-02-23 05:08:23.000
25573,25574,IgnStatus,True,2972,2972,1026305,2015-02-23 15:54:22.000,NaN,NaN,unknown,unknown,unknown,unknown,0,5246,0,True,1,NaN,1487,105369355,28.077361,-81.897083,2015-02-23 15:54:58.000


In [303]:
diagnostics_pivot = diagnostics.pivot(index=['Id', 'FaultId'], columns='Name', values='Value')

In [307]:
data = diagnostics_pivot.merge(faults_slim, left_on = 'FaultId', right_on = 'RecordID')

In [650]:
data = data.fillna(0)

In [666]:
data['EngineCoolantTemperature'] = data.EngineCoolantTemperature.str.replace(',', '.').astype(np.float64)

In [680]:
data['EngineCoolantTemperature'].dropna().describe()

count    586062.000000
mean        166.375531
std          36.045980
min         -23.800000
25%         165.200000
50%         183.200000
75%         186.800000
max         406.400000
Name: EngineCoolantTemperature, dtype: float64

In [684]:
data['EngineOilTemperature'] = data.EngineOilTemperature.str.replace(',', '.').astype(np.float64)

In [686]:
data['EngineOilTemperature'].dropna().describe()

count    583903.000000
mean        184.503887
std          91.812370
min         -40.000000
25%         171.668700
50%         201.087500
75%         213.743700
max        3212.544000
Name: EngineOilTemperature, dtype: float64

In [694]:
partial_data = (
    data
    .loc[data['EquipmentID']
    .isin(faulty_array)]
    .loc[data['spn'] == 1569]
)

In [698]:
partial_data['EngineCoolantTemperature'].describe()

count    2238.000000
mean      176.438338
std        27.490861
min        32.000000
25%       179.600000
50%       185.000000
75%       186.800000
max       224.600000
Name: EngineCoolantTemperature, dtype: float64

In [700]:
partial_data['EngineOilTemperature'].describe()

count    2233.000000
mean      195.749221
std        37.048495
min        32.112500
25%       194.843800
50%       208.906300
75%       215.993700
max       244.737500
Name: EngineOilTemperature, dtype: float64

In [704]:
derate_data = (
    data
    .loc[data['EquipmentID']
    .isin(faulty_array)]
    .loc[data['spn'] == 5246]
)

In [706]:
derate_data['EngineCoolantTemperature'].describe()

count    560.000000
mean     162.233214
std       42.047670
min       17.600000
25%      152.150000
50%      181.400000
75%      186.800000
max      213.800000
Name: EngineCoolantTemperature, dtype: float64

In [708]:
derate_data['EngineOilTemperature'].describe()

count    545.000000
mean     175.573227
std       50.313007
min       18.106250
25%      158.225000
50%      197.037500
75%      210.818800
max      228.706300
Name: EngineOilTemperature, dtype: float64

In [225]:
start = faults['EventTimeStamp'].iloc[0]
stop = faults['EventTimeStamp'].iloc[-1]

print(start, "\n", stop)

2015-02-21 10:47:13.000 
 2020-03-06 14:15:34.000


In [173]:
faults['activeTransitionCount'].unique()

array([  2, 127,   1, 126,  76,   5,  13,  22,   3,  14,  15,  36,  59,
        10,  42,   7,   8,  16,   9,   4,  24,  21,  43,   6,  61,  11,
        25,  20,  17,   0,  97,  98,  94,  12,  66,  48,  26,  30,  55,
        56,  23,  54,  18,  57,  58, 113,  32, 105,  68,  99, 100, 101,
       102,  95,  44, 104,  63,  19,  83,  60,  82,  52,  28,  29,  62,
        31, 114, 107,  87, 110,  71,  91, 108,  33,  39,  88,  69,  53,
       106,  64,  70,  92,  93,  27, 109,  65,  96,  67,  84,  40,  34,
        45,  35,  85,  37,  47,  51,  72,  38,  73,  74, 111,  75,  77,
        78,  49,  46, 103,  79,  80,  81,  41,  50, 119,  86, 120, 121,
       122,  89,  90, 112, 115, 123, 124, 116, 125, 117, 118], dtype=int64)

In [177]:
faults['active'].value_counts()

active
True     608454
False    578881
Name: count, dtype: int64

In [346]:
data = data.drop(['actionDescription', 'faultValue'], axis = 1)

In [682]:
(
    data
    .loc[data['EquipmentID']
    .isin(faulty_array)]
    .loc[data['spn'] == 1569]
    .groupby('EquipmentID')['EngineCoolantTemperature']
    .mean()
)

EquipmentID
105349576    194.0
105360462      NaN
105411041      NaN
105427203    165.2
1329           NaN
             ...  
2007         127.4
2009         179.6
2011         181.4
2021         181.4
306          183.2
Name: EngineCoolantTemperature, Length: 186, dtype: float64

In [354]:
sfc

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7119,Y,9996,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7120,Y,9997,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7121,Y,9998,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7122,Y,9999,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
